# Traitement des données de financements 

In [1]:
#importer les packages
import pandas as pd
import numpy as np
from project.server.main.utils import replace_all, get_scanR_structure, get_id, strip_outer_quotes
from project.server.main.cached_data_handler import get_structure, get_person
from project.server.main.features_into_dictionnary import persons, projects, address
from project.server.main.id_from_orcid import orcid_to_idref
from tqdm import tqdm
import pprint as pp
tqdm.pandas()
from project.server.main.my_pickle import load_cache,write_cache
import os
from dotenv import load_dotenv
import requests

load_dotenv()

Authorization = os.getenv('Authorization_access_185.XX')
Authorization_ORCID = os.getenv('Authorization_cluster_BSO_ORCID')
url_cluster = os.getenv('url_cluster')

In [2]:
sources=pd.read_json('sources.json')

In [3]:
source=list(sources.keys())[1]
source

'ANSES'

In [ ]:
#cache structures, personnes et orcid avec differentes sources de donnees
cached_data = {}
try:
    cached_data = load_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
except:
    write_cache(cached_data,f"./DATA/{source}/caches/cached_{source.lower()}_data.pkl")
    
cached_data_persons = {}
try:
    cached_data_persons = load_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
except:
    write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")
    
cached_data_orcid = {}
try:
    cached_data_orcid = load_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
except:
    write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")

# Données partenaires

In [ ]:
if source == 'SIRANO':
    pass
elif source == 'REG_IDF':
    response = requests.get(
        "https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/dim_map_projets_finances",
        headers={"Accept":"/"},
    )
    datas = response.json()
    date_projects=datas['metas']['default']['modified']
    
    response = requests.get(
        "https://data.iledefrance.fr/api/explore/v2.1/catalog/datasets/dim_map_projets_finances_entites_partenariat",
        headers={"Accept":"/"},
    )
    datas = response.json()
    date_partners=datas['metas']['default']['modified']
else:
    response = requests.get(
        f"http://www.data.gouv.fr/api/1/datasets/?organization={sources[source]['id']}&format=json&q=dos",
        headers={"Accept":"/"},
    )
    datas = response.json()
    date_projects=[data for data in datas['data'][0]['resources'] if sources[source]['keyword_projects1'] in str(data['title']) !=-1 and sources[source]['keyword_projects2'] in str(data['title']) !=-1 ][0]['last_modified']
    date_partners=[data for data in datas['data'][0]['resources'] if sources[source]['keywords_partners1'] in str(data['title']) !=-1 and sources[source]['keywords_partners1'] in str(data['title']) !=-1][0]['last_modified']

nbr_page=int(requests.get(f'http://185.161.45.213/projects/participations?where={"project_type":"{source}"}&projection={"modified_at":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
list_ids=[]
for i in range(1,nbr_page+1):
    page=requests.get(f'http://185.161.45.213/projects/participations?where={"project_type":"{source}"}&projection={"modified_at":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['modified_at'])
max_date_partners=max(list_ids)

nbr_page=int(requests.get(f'http://185.161.45.213/projects/projects?where={"type":"{source}"}&projection={"modified_at":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
list_ids=[]
for i in range(1,nbr_page+1):
    page=requests.get(f'http://185.161.45.213/projects/projects?where={"type":"{source}"}&projection={"modified_at":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['modified_at'])
max_date_projects=max(list_ids)

#if true => update
print(max_date_partners<date_partners or max_date_projects<date_projects)

In [ ]:
# amener les partenaires depuis le site 
if source=='ANR':
    page_partenaires_10 = requests.get(sources[source]['url_partners']).json()
    colonnes_partenaires_10 = page_partenaires_10['columns']
    donnees_partenaires_10 = page_partenaires_10['data']
    df_partenaires=pd.DataFrame(data=donnees_partenaires_10,columns=colonnes_partenaires_10)
elif source=='ANSES':
    df_from_anses=pd.read_excel(sources[source]['url_partners'])
    df_partenaires=df_from_anses.applymap(strip_outer_quotes)
    df_partenaires['annee']=df_partenaires.apply(lambda row: "20"+str(row[sources[source]['code_projet']].split('-')[1])[-2:], axis=1)
elif source=='IRESP':
    df_partenaires1=pd.read_csv(sources[source]['url_partners1'] ,sep=";", encoding='UTF-8')
    df_partenaires2=pd.read_csv(sources[source]['url_partners2'] ,sep=";", encoding='UTF-8')
    df_partenaires=pd.concat([df_partenaires1,df_partenaires2])
elif source=='ADEME':
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=",", encoding='ISO-8859-1', on_bad_lines='skip')
else:    
    df_partenaires=pd.read_csv(sources[source]['url_partners'] ,sep=";", encoding='ISO-8859-1')

df_partenaires=df_partenaires.reset_index()
del df_partenaires['index']

# Matcher établissement

In [ ]:
id_struct=df_partenaires
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
id_struct=id_struct.drop_duplicates(subset=[f"{sources[source]['nom_structure']}2"])

In [ ]:
import requests
from project.server.main.Pydref import Pydref
from retry import retry
import os
from dotenv import load_dotenv

load_dotenv()

#urls
url='https://affiliation-matcher.staging.dataesr.ovh/match'
url_cluster = os.getenv('url_cluster')

#get the structure id from the structure name
@retry(delay=200, tries=30000)
def get_structure(row,source,cached_data,nom_structure,pays,ville,code_projet,annee_input):
    if row[nom_structure] in list(cached_data.keys()):
        pass
    else:
        url='https://affiliation-matcher.staging.dataesr.ovh/match'
        f= ' '.join([str(row[y]) for y in [x for x in [nom_structure,ville,pays] if x in list(row.keys())]])
        print(f)
        if source=='REG_IDF':
            rnsr=requests.post(url, json= {"type":"rnsr",
                                           "year":str(row[code_projet].split('-')[3]),
                                           "query":f,"verbose":False})
        elif source not in ['REG_IDF','ADEME','SIRANO']:
            rnsr=requests.post(url, json= {"type":"rnsr",
                                           "year":"20"+str(row[code_projet].split('-')[1])[-2:],
                                           "query":f,"verbose":False})
        else:
            if source=='ADEME':
                annee=row['Date de dÃ©but du projet'][:4]
            elif source=='SIRANO':
                annee=row['annee_de_selection']
            else:
                annee=row[annee_input]
            rnsr=requests.post(url, json= {"type":"rnsr","year":str(annee),"query":f,"verbose":False})
        ror=requests.post(url, json= { "query" : f , "type":"ror"})
        grid=requests.post(url, json= { "query" : f , "type":"grid"})
        result_rnsr=rnsr.json()['results']
        result_ror=ror.json()['results']
        result_grid=grid.json()['results'] 
        print(result_rnsr,result_ror,result_grid)
        if result_rnsr != []:
            cached_data[row[nom_structure]]=result_rnsr
        elif result_rnsr != [] and result_grid != []:
            cached_data[row[nom_structure]]=result_grid
        elif result_rnsr != [] and result_grid == [] and result_ror != []:
            cached_data[row[nom_structure]]=result_ror
        else:
            cached_data[row[nom_structure]]=None

In [ ]:
id_struct.progress_apply(lambda row: get_structure(row,source,cached_data,sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays'],sources[source]['code_projet'],sources[source]['annee']), axis=1) 
write_cache(cached_data,f"./DATA/{source}/caches/cached_{source}_data.pkl")

In [ ]:
len(cached_data)

In [ ]:
id_struct['id_structure_matcher']=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: cached_data[x])

In [ ]:
id_struct

In [ ]:
id_struct=id_struct.reset_index()
del id_struct['index']
id_struct.to_json(f"./DATA/{source}/df_partenaires.json")

In [ ]:
id_struct=pd.read_json(f"./DATA/{source}/df_partenaires.json")
id_struct=id_struct[[sources[source]['nom_structure'],'id_structure_matcher']]
id_struct[f"{sources[source]['nom_structure']}2"]=id_struct.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))

df_partenaires[f"{sources[source]['nom_structure']}2"]=df_partenaires.loc[:,sources[source]['nom_structure']].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
df_partenaires_struct=pd.merge(df_partenaires,id_struct[[f"{sources[source]['nom_structure']}2",'id_structure_matcher']], on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#compléter les données avec scanR
url_scanr='https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/scanR/projects.json'
requete_scanR = requests.get(url_scanr)
page_scanR= requete_scanR.json()
df_scanR=pd.DataFrame(page_scanR)
scanR=df_scanR.explode('participants').loc[:,['id','participants']]
scanR=scanR.rename(columns={'id':'id_anr'})
scanR['index']=[x for x in range(len(scanR))]
scanR=scanR.set_index('index')
scanR['id_structure_scanr']=scanR['participants'].apply(lambda x: x.get(str('structure')) if isinstance(x, dict) else None )
scanR['nom_struct']=scanR['participants'].apply(lambda x: get_scanR_structure(x))
del scanR['participants']
scanR_nettoye=scanR.drop_duplicates(subset='nom_struct')
scanR_nettoye[f"{sources[source]['nom_structure']}2"]=scanR_nettoye.loc[:,'nom_struct'].apply(lambda x: replace_all(str(x).lower()))
scanR_nettoye=scanR_nettoye[['id_structure_scanr',f"{sources[source]['nom_structure']}2"]]
scanR_nettoye=scanR_nettoye.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")

In [ ]:
df_partenaires_struct=pd.merge(df_partenaires_struct,scanR_nettoye, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_struct

In [ ]:
#######fichier avec les identifiants structures rettrouvés à la main par Emmanuel ==> 'code'
scanr_structures=pd.read_excel('scanr_partenaires_non_identifies.xlsx')
scanr_structures[f"{sources[source]['nom_structure']}2"]=scanr_structures.loc[:,'Nom'].apply(lambda x: replace_all(str(x).lower().replace(" d e"," d'e").replace(" d a"," d'a").replace(" d i"," d'i").replace(" d o"," d'o").replace(" d u"," d'u").replace(" d y"," d'y").replace(" d h"," d'h").replace(" l e"," l'e").replace(" l a"," l'a").replace(" l i"," l'i").replace(" l o"," l'o").replace(" l u"," l'u").replace(" l y"," l'y").replace(" l h"," l'h")))
scanr_structures=scanr_structures[[f"{sources[source]['nom_structure']}2",'code']]
scanr_structures=scanr_structures.dropna().drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
df_partenaires_complet=pd.merge(df_partenaires_struct,scanr_structures, on=f"{sources[source]['nom_structure']}2", how='left')
df_partenaires_complet

In [ ]:
if 'finess' in list(df_partenaires.columns):
    finess_siret=pd.read_json("finess_siret-siege.json")
    df_partenaires_complet=pd.merge(df_partenaires_complet,finess_siret,how='left', on='finess')

In [ ]:
df_partenaires_complet[sources[source]['identifiants_preferes_structure']]

In [ ]:
df_partenaires_complet['id_structure']=df_partenaires_complet.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_structure']), axis=1)
df_partenaires_complet

In [ ]:
#df_partenaires_complet[pd.isna(df_partenaires_complet.id_structure)]
df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]

In [ ]:
#df_partenaires_complet.to_excel(f"./DATA/{source}/df_partenaires_id_structures.xlsx")
df_partenaires_complet.to_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
df_partenaires_complet=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
df_partenaires_complet

In [ ]:
########récupération des structures sans identifiants pour les donner à Emmanuel
identifiants_a_remplir=df_partenaires_complet.loc[(pd.isna(df_partenaires_complet['id_structure']))|(str(df_partenaires_complet['id_structure'])=='None')|(str(df_partenaires_complet['id_structure'])=='nan')]
identifiants_a_remplir
identifiants_a_remplir=identifiants_a_remplir.drop_duplicates(subset=f"{sources[source]['nom_structure']}2")
identifiants_a_remplir=identifiants_a_remplir.reset_index()
del identifiants_a_remplir['index']

In [ ]:
identifiants_a_remplir['Projet.Partenaire.Organisme'][0]

In [ ]:
if sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] in list(identifiants_a_remplir.columns) and sources[source]['adresse'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['adresse'],sources[source]['ville'],sources[source]['pays']]]
elif sources[source]['region'] in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['region']]]
elif sources[source]['ville'] in list(identifiants_a_remplir.columns) and sources[source]['pays'] not in list(identifiants_a_remplir.columns):
    identifiants_a_remplir=identifiants_a_remplir[[sources[source]['nom_structure'],sources[source]['ville']]]
identifiants_a_remplir

In [ ]:
identifiants_a_remplir.to_excel(f"./missing_ids_structures/partenaires_non_identifies_{source}.xlsx", index=False)

# Matcher des chercheurs

In [ ]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [ ]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)
    #df_partenaires.to_excel(f"./DATA/{source}/df_partners_id_personne.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partners_id_person.json")

In [ ]:
len(cached_data_persons)

In [ ]:
write_cache(cached_data_persons,f"./DATA/{source}/caches/cached_{source.lower()}_data_persons.pkl")

In [ ]:
df_partenaires['id_personne']=df_partenaires.progress_apply(lambda row: get_person(row, cached_data_persons,sources[source]['nom'],sources[source]['prenom']), axis=1)

In [ ]:
df_partenaires.to_json(f"./DATA/{source}/df_partners_id_person.json")

In [ ]:
if sources[source]['id_ORCID'] in list(df_partenaires.columns):
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person.json")
    df_partenaires['idref_ORCID']=df_partenaires.progress_apply(lambda row: orcid_to_idref(row,cached_data_orcid,sources[source]['id_ORCID'],Authorization_ORCID), axis=1)
    #write_cache(cached_data_orcid,f"./DATA/{source}/caches/cached_{source.lower()}_data_orcid.pkl")
    #df_partenaires.to_excel(f"./DATA/{source}/df_partners_id_person_ORCID.xlsx")
    df_partenaires.to_json(f"./DATA/{source}/df_partners_id_person_ORCID.json")

In [ ]:
len(cached_data_orcid)

In [ ]:
df_partenaires

# ENVOI DES PROJETS SUR SCANR

In [4]:
if len(sources[source]['identifiants_preferes_personne'])>=2:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person_ORCID.json")
elif len(sources[source]['identifiants_preferes_personne'])==1:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_person.json")
else:
    df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [5]:
df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure']=df_partenaires.loc[df_partenaires.id_structure.apply(lambda x :isinstance(x,list)),'id_structure'].apply(lambda y: y[0])

In [6]:
if len([x for x in ['nom', 'prenom'] if x in list(sources[source].keys())])==2:
    df_partenaires['id_person']=df_partenaires.apply(lambda row: get_id(row,sources[source]['identifiants_preferes_personne']), axis=1)
    df_partenaires['persons']=df_partenaires.progress_apply(lambda row: persons(row,sources[source]['prenom'],sources[source]['nom']) ,axis=1)
else:
    df_partenaires['persons']=np.nan

100%|██████████| 1856/1856 [00:00<00:00, 31941.78it/s]


In [7]:
if source != 'SIRANO':
    df_partenaires=df_partenaires.groupby([sources[source]['code_projet']]).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False).reset_index()
else:
    df_projets=df_partenaires.groupby([sources[source]['code_projet'], sources[source]['annee'], sources[source]['acronyme'],sources[source]['titre'],sources[source]['budget']], dropna=False).agg({'persons': lambda x: [ y for y in x.tolist() if pd.isna(y)==False]}, dropna=False)

In [8]:
# amener les projets depuis le site 
if source=='ANR':
    page_projets_10 = requests.get(sources[source]['url_projects']).json()
    colonnes_projets_10 = page_projets_10['columns']
    donnees_projets_10 = page_projets_10['data']
    df_projets=pd.DataFrame(data=donnees_projets_10,columns=colonnes_projets_10)
elif source=='IRESP':
    df_projets1=pd.read_csv(sources[source]['url_projects1'] ,sep=";", encoding='UTF-8')
    df_projets2=pd.read_csv(sources[source]['url_projects2'] ,sep=";", encoding='UTF-8')
    df_projets=pd.concat([df_projets1,df_projets2])
    df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet_FR']=df_projets.loc[pd.isna(df_projets['Titre_du_projet_FR']),'Titre_du_projet']
elif source=='ANSES':
    df_projets=pd.read_excel(sources[source]['url_projects'])
    df_projets=df_projets.applymap(strip_outer_quotes)
elif source!='SIRANO':
    df_projets=pd.read_csv(sources[source]['url_projects'] ,sep=";", encoding='ISO-8859-1')

df_projets=df_projets.reset_index()
del df_projets['index']

C:\Users\haallat\AppData\Local\Temp\ipykernel_12656\2316314121.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_projets=df_projets.applymap(strip_outer_quotes)


In [9]:
if source!='SIRANO':
    df_projets=pd.merge(df_projets,df_partenaires,on=sources[source]['code_projet'], how='left')
else :
    df_projets['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[sources[source]['annee']]}-{row[sources[source]['acronyme']]}" , axis=1)
    del df_projets['code_projet']
    sources[source]['code_projet']='id'

df_projets['type']=source
df_projets['name']=df_projets.progress_apply(lambda row: projects(row,sources[source]['titre_fr'],sources[source]['titre_en']) ,axis=1)
df_projets['description']=df_projets.progress_apply(lambda row: projects(row,sources[source]['resume_fr'],sources[source]['resume_en']) ,axis=1)

100%|██████████| 630/630 [00:00<00:00, 27919.78it/s]


In [19]:
df_projets.loc[:,sources[source]['budget']]=df_projets.loc[:20,sources[source]['budget']].apply(lambda x : int(str(x).replace('.0','').replace('.00','').replace(' ','').replace(',','.').replace('€','').replace('\x80','')) if str(x).find('.0')!=-1 or str(x).find(",0")==-1 else float(str(x).replace(' ','').replace(',','.').replace('€','').replace('\x80','')))
df_projets=df_projets.rename(columns={sources[source]['annee']:'year',sources[source]['acronyme']:'acronym',
                                      sources[source]['budget']:'budget_financed',sources[source]['code_projet']:'id'})
df_projets=df_projets[['id','type','name','description','acronym','year','budget_financed','persons']]

In [20]:
dict_row=df_projets.iloc[1,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

{'id': 'ANSES-06-EST-003',
 'type': 'ANSES',
 'name': {'fr': "Développement d'outils de surveillance biologique pour l'évaluation des risques à la santé de travailleurs en arboriculture et en viticulture exposés aux fongicides",
  'en': 'Development of biomarkers to assess exposure and risks of agriculture workers exposed to fongicides'},
 'description': {'fr': 'Un grand nombre d’études épidémiologiques indiquent de manière cohérente une augmentation de la morbidité/mortalité cardiaque et pulmonaire chez l’homme lors de l’exposition aux particules de l’air (PM). Le mécanisme expliquant les effets toxiques des PM sur la santé ne sont que partiellement compris mais une hypothèse actuellement avancée est que le stress oxidatif serait à la base de beaucoup des effets observés. Le stress oxidatif serait initié par la formation d’espèces réactives de l’oxygène (ROS) au niveau des cellules affectées. Bien qu’il y ait encore beaucoup de discussions concernant les composés qui sont responsables

In [ ]:
#envoi
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [ ]:
pd.Series([x.get('issues').get('id') for x in err]).drop_duplicates().tolist() 

pour mettre à jour

In [21]:
###AAATTTENTION CHANGER A CHAQUE FOIS LE TYPE
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"IRESP"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={"type":"IRESP"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_projets['id']) if x not in list_ids]

projets_a_retirer=[x for x in list_ids if x not in list(df_projets['id'])]

df_projets = df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)]

page 1
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183

In [22]:
len(projets_a_ajouter)

630

In [23]:
len(df_projets[df_projets['id'].apply(lambda x: x in projets_a_ajouter)])

630

In [24]:
len(df_projets[df_projets['id'].apply(lambda x: x in projets_a_retirer)])

0

In [25]:
dict_row=df_projets.iloc[0,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
dict_row2

{'id': 'ANSES-06-EST-002',
 'type': 'ANSES',
 'name': {'fr': 'Projet PAISIM : Pollution Atmosphérique, Inégalités Sociales et Infarctus du Myocarde',
  'en': 'The PAISIM project: Air Pollution, Social Inequalities & Myocardial Infarction'},
 'description': {'fr': 'Depuis une quinzaine d’années, plusieurs équipes se sont intéressées aux liens existant entre le polymorphisme de l’ALAD G177C (acide delta-aminolévulinique déshydratase) (rs1800435) et l’intoxication par le plomb. Les études publiées ne sont pas concordantes quant aux liens existants entre ce polymorphisme et les effets du plomb sur la fonction rénale. L’opportunité nous est donnée d’analyser ces liens sur une cohorte d’ex-salariés d’une fonderie de plomb et de zinc de la Région Nord – Pas de Calais, qui bénéficient depuis 2003 d’un suivi clinique et biologique post-expositionnel. Ce suivi va être prolongé jusqu’en 2009, dans le cadre d’une étude scientifique visant à étudier les facteurs de variation de la toxicocinétique d

In [26]:
err=[]
for i,row in df_projets.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x')&(str(v)!='[]'))}
    try:
       r=requests.post('http://185.161.45.213/projects/projects', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

In [27]:
len(projets_a_ajouter)

630

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/projects?where={"type":"ANR"}&projection={"id":1,"year":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])
nbr_page

In [ ]:
list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/projects?where={%22type%22:%22ANR%22}&projection={%22id%22:1,%22year%22:1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        list_ids.append(page['data'][k]['year'])

In [ ]:
max(list_ids)

In [ ]:
df_projets = df_projets[df_projets['id'].apply(lambda x: x in list_ids)]

In [ ]:
"""
en principe, il faut que je mette à jour tous les projets 
qui ne sont pas clos avec les titres résumés et persons, mais 
peut etre qu'on peut mettre à jour automatiquement les projets
des 3 dernières années ? 

"""

err=[]
for id in df_projets.iterrows() :
    url = f"http://185.161.45.213/projects/projects/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"year": 2023}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)


# ENVOI DES PARTENAIRES SUR SCANR

In [28]:
df_partenaires=pd.read_json(f"./DATA/{source}/df_partners_id_structures.json")

In [29]:
df_partenaires.columns

Index(['Projet.Code_initial', 'Projet.Code', 'Projet.Acronyme',
       'Projet.Responsable.Nom', 'Projet.Responsable.Prénom',
       'Projet.Partenaire.numero', 'Projet.Partenaire.Organisme',
       'Projet.Partenaire.Codepostal', 'Projet.Partenaire.Ville',
       'Projet.Partenaire.Pays', 'annee', 'Projet.Partenaire.Organisme2',
       'id_structure_matcher', 'id_structure_scanr', 'code', 'id_structure'],
      dtype='object')

In [30]:
### ATTENTION, vérifier que les projets sirano sont dans des structures françaises
if source=='IRESP':
    df_partenaires[sources[source]['pays']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[1].replace(')','') if x.find('(')>=0 else 'France')
    df_partenaires.loc[:,sources[source]['ville']]=df_partenaires.loc[:,sources[source]['ville']].apply(lambda x: x.split('(')[0] if x.find('(')>=0 else x)
  
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires.loc[:,'id_structure']=df_partenaires.loc[:,'id_structure'].apply(lambda x: x[0] if isinstance(x,list) else x )

In [31]:
df_partenaires

,Projet.Code_initial,Projet.Code,Projet.Acronyme,Projet.Responsable.Nom,Projet.Responsable.Prénom,Projet.Partenaire.numero,Projet.Partenaire.Organisme,Projet.Partenaire.Codepostal,Projet.Partenaire.Ville,Projet.Partenaire.Pays,annee,Projet.Partenaire.Organisme2,id_structure_matcher,id_structure_scanr,code,id_structure,address
0,2006/1/2,ANSES-06-EST-002,PAISIM,Bard,Denis,1,LERES Laboratoire d'Etude et de Recherche en e...,35403,Rennes,France,2006,lereboratoirurechercheenenvironnementante,None,None,None,None,"{'city': 'Rennes', 'country': 'France'}"
1,2006/1/2,ANSES-06-EST-002,PAISIM,Arveiler,Dominique,2,Registre des cardiopathies ischémiques du Bas-...,67000,Strasbourg,France,2006,regitrecardiopathieichemiquebarhin,None,None,e1S12,e1S12,"{'city': 'Strasbourg', 'country': 'France'}"
2,2006/1/2,ANSES-06-EST-002,PAISIM,Eilstein,Daniel,3,Département Santé Environnement,94415,Saint Maurice,France,2006,ementanteenvironnement,None,None,X3DyA,X3DyA,"{'city': 'Saint Maurice', 'country': 'France'}"
3,2006/1/3,ANSES-06-EST-003,,Droz,Olivier,1,IST Institut universitaire romand de Santé au ...,,Lausanne,Suisse,2006,itintitutuniveritaireromandanteautravail,None,None,04mcdza51,04mcdza51,"{'city': 'Lausanne', 'country': 'Suisse'}"
4,2006/1/3,ANSES-06-EST-003,,Bouchard,Michèle,2,Département de santé environnementale et de sa...,,Montréal,Québec Canada,2006,ementanteenvironnementaleanteautravail,None,None,0161xgx34,0161xgx34,"{'city': 'Montréal', 'country': 'Québec Canada'}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,ANSES-2024-RF-10,ANSES-24-RF-10,MAELSTROEM,Bories,Serge,1,Cea Grenoble LAPCI,38054,Grenoble,France,2024,ceagrenoblepci,None,None,None,None,"{'city': 'Grenoble', 'country': 'France'}"
1852,ANSES-2024-RF-10,ANSES-24-RF-10,MAELSTROEM,Deltour,Isabelle,2,Centre International de Recherche sur le Cance...,69378,Lyon,France,2024,centreinternationalrechercheurcancer\nectionen...,None,None,None,None,"{'city': 'Lyon', 'country': 'France'}"
1853,ANSES-2024-RF-10,ANSES-24-RF-10,MAELSTROEM,Wiart,Joe,3,Telecom ParisTech Chaire C2M,75013,Paris,France,2024,telecomparitechchairec2m,None,None,None,None,"{'city': 'Paris', 'country': 'France'}"
1854,ANSES-2024-RF-13,ANSES-24-RF-13,CORSICA,Puginier Pinet,Emilie,1,Université Bordeaux 1 Laboratoire IMS UMR5218 ...,33405,Talence,France,2024,univeriteboraux1boratoireimumr5218cnr,[200711887V],None,None,200711887V,"{'city': 'Talence', 'country': 'France'}"


In [32]:
if source in ['ANSES','SIRANO']:
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row[sources[source]['nom']]}-{row[sources[source]['prenom']]}" , axis=1)
if source =='REG_IDF':
    df_partenaires['id']=df_partenaires.apply(lambda row: f"{row[sources[source]['code_projet']]}-{row[str(sources[source]['nom_structure'])+'2']}-{row['entite_role']}" , axis=1)
df_partenaires['address']=df_partenaires.apply(lambda row: address(row,sources[source]['pays'],sources[source]['ville'],source), axis=1)
df_partenaires=df_partenaires.rename(columns={sources[source]['nom_structure']: 'name', sources[source]['code_projet']: 'project_id', 'id_structure':'participant_id','Projet.Partenaire.Code_Decision_ANR':'id'})
df_partenaires=df_partenaires[['name','id','project_id','participant_id','address']]
df_partenaires['project_type']=source
df_partenaires['participant_id']=df_partenaires.loc[:,'participant_id'].apply(lambda x: str(x[0]).replace('.0','') if isinstance(x,list) else str(x).split(';')[0].replace('.0',''))
df_partenaires=df_partenaires[['id','project_id', 'project_type', 'participant_id', 'name','address']]
df_partenaires['name'] = df_partenaires['name'].astype(str)
df_partenaires

,id,project_id,project_type,participant_id,name,address
0,ANSES-06-EST-002-lereboratoirurechercheenenvir...,ANSES-06-EST-002,ANSES,None,LERES Laboratoire d'Etude et de Recherche en e...,"{'city': 'Rennes', 'country': 'France'}"
1,ANSES-06-EST-002-regitrecardiopathieichemiqueb...,ANSES-06-EST-002,ANSES,e1S12,Registre des cardiopathies ischémiques du Bas-...,"{'city': 'Strasbourg', 'country': 'France'}"
2,ANSES-06-EST-002-ementanteenvironnement-Eilste...,ANSES-06-EST-002,ANSES,X3DyA,Département Santé Environnement,"{'city': 'Saint Maurice', 'country': 'France'}"
3,ANSES-06-EST-003-itintitutuniveritaireromandan...,ANSES-06-EST-003,ANSES,04mcdza51,IST Institut universitaire romand de Santé au ...,"{'city': 'Lausanne', 'country': 'Suisse'}"
4,ANSES-06-EST-003-ementanteenvironnementaleante...,ANSES-06-EST-003,ANSES,0161xgx34,Département de santé environnementale et de sa...,"{'city': 'Montréal', 'country': 'Québec Canada'}"
...,...,...,...,...,...,...
1851,ANSES-24-RF-10-ceagrenoblepci-Bories-Serge,ANSES-24-RF-10,ANSES,None,Cea Grenoble LAPCI,"{'city': 'Grenoble', 'country': 'France'}"
1852,ANSES-24-RF-10-centreinternationalrechercheurc...,ANSES-24-RF-10,ANSES,None,Centre International de Recherche sur le Cance...,"{'city': 'Lyon', 'country': 'France'}"
1853,ANSES-24-RF-10-telecomparitechchairec2m-Wiart-Joe,ANSES-24-RF-10,ANSES,None,Telecom ParisTech Chaire C2M,"{'city': 'Paris', 'country': 'France'}"
1854,ANSES-24-RF-13-univeriteboraux1boratoireimumr5...,ANSES-24-RF-13,ANSES,200711887V,Université Bordeaux 1 Laboratoire IMS UMR5218 ...,"{'city': 'Talence', 'country': 'France'}"


In [33]:
df_partenaires[df_partenaires.duplicated(subset=['id'])]

,id,project_id,project_type,participant_id,name,address


In [34]:
dict_row=df_partenaires.iloc[1,:].to_dict()
dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
dict_row2

{'id': 'ANSES-06-EST-002-regitrecardiopathieichemiquebarhin-Arveiler-Dominique',
 'project_id': 'ANSES-06-EST-002',
 'project_type': 'ANSES',
 'participant_id': 'e1S12',
 'name': 'Registre des cardiopathies ischémiques du Bas-Rhin',
 'address': {'city': 'Strasbourg', 'country': 'France'}}

In [35]:
len(df_partenaires)

1856

In [ ]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Mise à jour

In [36]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500&page=1', headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANSES"}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])
    
projets_a_ajouter=[x for x in list(df_partenaires['id'].drop_duplicates()) if x not in list(pd.Series(list_ids).drop_duplicates())]

projets_a_retirer=[x for x in list_ids if x not in list(df_partenaires['id'])]

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in projets_a_ajouter)]


page 1
k 0
k 1
k 2
k 3
k 4
k 5
k 6
k 7
k 8
k 9
k 10
k 11
k 12
k 13
k 14
k 15
k 16
k 17
k 18
k 19
k 20
k 21
k 22
k 23
k 24
k 25
k 26
k 27
k 28
k 29
k 30
k 31
k 32
k 33
k 34
k 35
k 36
k 37
k 38
k 39
k 40
k 41
k 42
k 43
k 44
k 45
k 46
k 47
k 48
k 49
k 50
k 51
k 52
k 53
k 54
k 55
k 56
k 57
k 58
k 59
k 60
k 61
k 62
k 63
k 64
k 65
k 66
k 67
k 68
k 69
k 70
k 71
k 72
k 73
k 74
k 75
k 76
k 77
k 78
k 79
k 80
k 81
k 82
k 83
k 84
k 85
k 86
k 87
k 88
k 89
k 90
k 91
k 92
k 93
k 94
k 95
k 96
k 97
k 98
k 99
k 100
k 101
k 102
k 103
k 104
k 105
k 106
k 107
k 108
k 109
k 110
k 111
k 112
k 113
k 114
k 115
k 116
k 117
k 118
k 119
k 120
k 121
k 122
k 123
k 124
k 125
k 126
k 127
k 128
k 129
k 130
k 131
k 132
k 133
k 134
k 135
k 136
k 137
k 138
k 139
k 140
k 141
k 142
k 143
k 144
k 145
k 146
k 147
k 148
k 149
k 150
k 151
k 152
k 153
k 154
k 155
k 156
k 157
k 158
k 159
k 160
k 161
k 162
k 163
k 164
k 165
k 166
k 167
k 168
k 169
k 170
k 171
k 172
k 173
k 174
k 175
k 176
k 177
k 178
k 179
k 180
k 181
k 182
k 183

In [37]:
len(projets_a_ajouter)

1011

In [39]:
len(projets_a_retirer)

892

In [ ]:
len(df_partenaires)

In [ ]:
err=[]
for i,row in df_partenaires.iterrows():
    dict_row=row.to_dict()
    dict_row2={k:v for k,v in list(dict_row.items()) if ((str(v)!='nan')&(str(v)!='NaN')&(str(v)!='None')&(str(v)!='x'))}
    try:
       r=requests.post('http://185.161.45.213/projects/participations', json = dict_row2, headers={"Authorization":Authorization})
       res= r.json()
       if res.get('status')=='ERR':
           print(i)
           err.append(res)
           if res.get('error').get('code')!=422:
               print(err)
               pp.pprint(err)
    except Exception as e:
        pp.pprint(e)

Modifications

In [ ]:
nbr_page=int(requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={1}", headers={"Authorization":Authorization}).json()['hrefs']['last']['href'].split('page=')[1])

list_ids=[]
for i in range(1,nbr_page+1):
    print("page",i)
    page=requests.get('http://185.161.45.213/projects/participations?where={"project_type":"ANR","participant_id":{"$exists":false}}&projection={"id":1}&max_results=500'+f"&page={i}", headers={"Authorization":Authorization}).json()
    for k in range(len(page['data'])):
        print("k",k)
        list_ids.append(page['data'][k]['id'])

df_partenaires = df_partenaires[df_partenaires['id'].apply(lambda x: x in list_ids)]

In [ ]:
len(list_ids)

In [ ]:
df_partenaires=df_partenaires[(df_partenaires.participant_id!='x') & (df_partenaires.participant_id!='None')]

In [ ]:
df_partenaires

In [ ]:
err=[]
for i,row in df_partenaires.iterrows() :
    print(i)
    id=row['id']
    url = f"http://185.161.45.213/projects/participations/{id}"
    project=requests.get(url, headers={'Authorization': Authorization}).json()
    head = {"Authorization": Authorization, "If-Match": project['etag'], "Content-Type": "application/json"}

    r = requests.patch(url, json = {"participant_id": row['participant_id']}, headers=head)
    res= r.json()
    if res.get('status')=='ERR':
        err.append(res)
        if res.get('error').get('code')!=422:
            print(err)
            pp.pprint(err)

In [ ]:
pd.Series([x.get('issues').get('id')[25:] for x in err]).drop_duplicates().tolist()

Faire un nouveau fichier avec toutes les structures manquantes

In [ ]:
struct_manq=pd.read_excel('scanr_partenaires_non_identifies.xlsx')

In [ ]:
nom1=struct_manq[struct_manq.New=='IRESP'].drop_duplicates(subset='Nom').Nom

In [ ]:
iresp_actuel=pd.read_excel('./missing_ids_structures/partenaires_non_identifies_IRESP.xlsx')

In [ ]:
nom2=iresp_actuel['Nom_equipe']

In [ ]:
pd.Series(list(nom1)+list(nom2))

In [ ]:
pd.Series(list(nom1)+list(nom2)).drop_duplicates()